In [13]:
import time
import os

In [2]:
from dataclasses import dataclass, field
from enum import Enum
from requests import Session

In [16]:
import pickle
from xml.etree import ElementTree as ET
from typing import List, Tuple, Dict

In [4]:
import numpy as np
import pandas as pd

In [5]:
from tqdm import tqdm

---

In [17]:
@dataclass
class BGGElement:
    id: str


@dataclass
class Thread(BGGElement):
    subject: str
    author: str
        
    posts: List[Tuple[str, str]] = field(repr=False)
        
@dataclass
class Forum(BGGElement):
    title: str
        
    threads: List[Thread] = field(repr=False)    

---

In [18]:
@dataclass
class Thing:
    kind = 'thing'
    
    id: str
     
    
@dataclass
class ForumContainer:
    _forumlist: Dict[str, Forum] = field(init=False, repr=False, default_factory=dict)
        
    def __getitem__(self, title):
        return self._forumlist[title]
    
    def __setitem__(self, title, forum):
        if not isinstance(forum, Forum):
            raise RuntimeError('non-Forum passed')
        self._forumlist[title] = forum
        
    def load_forumlist(self, forumlist):
        for forum in forumlist:
            self[forum.title] = forum
        
    def titles(self):
        return self._forumlist.keys()
    
    def forums(self):
        return self._forumlist.values()
    
    def items(self):
        return self._forumlist.items()
    
    def __iter__(self):
        return iter(self._forumlist)


@dataclass
class BoardGame(Thing, ForumContainer):
    kind = 'boardgame'
    
    name: str
    description: str = field(repr=False)
    image: str = field(repr=False)
    yearpublished: int
        
    minage: int = field(repr=False)
        
    minplaytime: int
    maxplaytime: int
        
    minplayers: int
    maxplayers: int
    nplayers_best: str
        
    categories: List[str] = field(repr=False)
        
    def __str__(self):
        return f'''
        {self.image}
        {self.name} ({self.yearpublished})
        R: {self.minage}+
        
        {self.description}
        
        Playtime: {self.minplaytime} - {self.maxplaytime}
        Players: {self.minplayers} - {self.maxplayers} ({self.nplayers_best})
        
        {self.categories}
        '''
    
    @staticmethod
    def from_xml(item_xml):
        id_ = item_xml.get('id')
        attrs = dict()

        for attr in (
            ('name', str), ('description', str),
            ('yearpublished', int),
            ('minplaytime', int), ('maxplaytime', int),
            ('minplayers', int), ('maxplayers', int),
            ('minage', int),
        ):
            attr, cast = attr
            attrs[attr] = cast(item_xml.find(attr).get('value'))


        for attr in (
            ('image', str),
            ('description', str),
        ):
            attr, cast = attr
            attrs[attr] = cast(item_xml.find(attr).text)


        nplayers_tag = find_tag_by_attr(item_xml, 'poll', 'name', 'suggested_numplayers')
        nplayers_poll = [
            (
                result.get('numplayers'),
                int(find_tag_by_attr(result, 'result', 'value', 'Best').get('numvotes'))
            )
            for result in nplayers_tag
        ]
        attrs['nplayers_best'] = sorted(nplayers_poll, key=lambda x: x[1]).pop()[0]


        attrs['categories'] = [
            link.get('value')
            for link in find_tags_by_attr(item_xml, 'link', 'type', 'boardgamecategory')
        ]

        return BoardGame(id_, **attrs)
        
        
@dataclass(frozen=True)
class User:
    login: str

---

In [19]:
def find_tags_by_attr(xml_tree, tag, attr_key, attr_value):
    return [
        cand for cand in xml_tree.findall(tag)
        if cand.get(attr_key) == attr_value
    ]

In [20]:
def find_tag_by_attr(xml_tree, tag, attr_key, attr_value):
    return find_tags_by_attr(xml_tree, tag, attr_key, attr_value).pop()

---

In [21]:
BGG_ROOT_PATH = 'https://boardgamegeek.com/xmlapi2/'

FORUMLIST_SLEEP = 5
THREAD_SLEEP = 2


class BggSession(Session):
    def get_things(self, thing_class, ids):
        request_params = {
            'id': ','.join(ids),
            'type': thing_class.kind,
            'pagesize': 100,
        }
        response = self.get(BGG_ROOT_PATH + 'thing', params=request_params)
        if not response.ok:
            raise RuntimeError(response.content)
        
        return [thing_class.from_xml(item) for item in ET.fromstring(response.content)]
    
    def get_hotest(self, thing_class):
        response = self.get(BGG_ROOT_PATH + 'hot', params={'type': thing_class.kind})
        if not response.ok:
            raise RuntimeError(response.content)
        
        ids = [item.get('id') for item in ET.fromstring(response.content)]
            
        return self.get_things(thing_class, ids)
    
    def get_thread_posts(self, thread_id):
        response = self.get(BGG_ROOT_PATH + 'thread', params={'id': thread_id})
        if not response.ok:
            raise RuntimeError(response.content)

        tree = ET.fromstring(response.content)
        return [
            (post.get('username'), post.find('body').text)
            for post in tree.find('articles')
        ]
    
    def get_forum(self, forum_id):
        response = self.get(BGG_ROOT_PATH + 'forum', params={'id': forum_id})
        if not response.ok:
            raise RuntimeError(response.content)

        tree = ET.fromstring(response.content)
        threads = list()
        for thread in tree.find('threads'):
            time.sleep(THREAD_SLEEP)
            
            threads.append(Thread(
                thread.get('id'),
                subject=thread.get('subject'), author=thread.get('author'),
                posts=self.get_thread_posts(thread.get('id'))
            ))
        
        return Forum(forum_id, tree.get('title'), threads)
    
    def get_forumlist(self, thing_id):
        response = self.get(BGG_ROOT_PATH + 'forumlist', params={'type': 'thing', 'id': thing_id})
        if not response.ok:
            raise RuntimeError(response.content)
            
        tree = ET.fromstring(response.content)
        forums = list()
        for forum_tag in tqdm(tree, desc=f'Thing {thing_id} ForumList grep'):
            time.sleep(FORUMLIST_SLEEP)
            forums.append(self.get_forum(forum_tag.get('id')))
            
        return forums

---

In [22]:
SNAPSHOT_PATH = 'snapshot'

In [24]:
bgg_session = BggSession()

In [26]:
games = bgg_session.get_hotest(BoardGame)

for i, game in enumerate(games):
    try:
        forumlist = bgg_session.get_forumlist(game.id)
        game.load_forumlist(forumlist)

        with open(os.path.join(SNAPSHOT_PATH, game.id), 'wb') as output_stream:
            pickle.dump(game, output_stream)
        print(i, game.id, 'SUCCESS', end='')
    except BaseException as exc:
        print(i, game.id, 'FAILED')

Thing 291453 ForumList grep: 100%|██████████| 10/10 [02:08<00:00, 12.86s/it]


0 291453 SUCCESS

Thing 342942 ForumList grep: 100%|██████████| 10/10 [09:33<00:00, 57.40s/it]


1 342942 SUCCESS

Thing 363204 ForumList grep: 100%|██████████| 10/10 [00:59<00:00,  5.94s/it]


...

46 291457 SUCCESS

Thing 244521 ForumList grep:   0%|          | 0/10 [00:00<?, ?it/s]

47 255984 SUCCESS

Thing 244521 ForumList grep: 100%|██████████| 10/10 [09:02<00:00, 54.21s/it]

48 244521 SUCCESS

In [33]:
for i, game in enumerate([games[24], games[28], games[36], games[44]]):
    try:
        forumlist = bgg_session.get_forumlist(game.id)
        game.load_forumlist(forumlist)

        with open(os.path.join(SNAPSHOT_PATH, game.id), 'wb') as output_stream:
            pickle.dump(game, output_stream)
        print(i, game.id, 'SUCCESS', end='')
    except BaseException as exc:
        print(i, game.id, 'FAILED')

0 205637 FAILED


Thing 174430 ForumList grep:  80%|████████  | 8/10 [17:15<04:18, 129.46s/it]


1 174430 FAILED


Thing 352263 ForumList grep: 100%|██████████| 10/10 [02:20<00:00, 14.03s/it]


2 352263 SUCCESS

Thing 167355 ForumList grep:  30%|███       | 3/10 [06:53<16:04, 137.75s/it]

3 167355 FAILED


---

In [ ]:
users = list()
for game in games_trunc:
    for title, forum in game.items():
        for thread in forum.threads:
            users.extend(set([post[0] for post in thread.posts]))

In [ ]:
len(users)

In [ ]:
len(set(users))

In [ ]:
game.items()

---